In [1]:
%%writefile multiprocessing_test.py
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

Writing multiprocessing_test.py


In [2]:
%run multiprocessing_test.py

[1, 4, 9]


In [6]:
%%writefile multiprocessing_test.py
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

Overwriting multiprocessing_test.py


In [7]:
%%writefile multiprocessing_test.py
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

Overwriting multiprocessing_test.py


In [9]:
%%writefile multiprocessing_test.py
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    mp.set_start_method('spawn')
    q = mp.Queue()
    p = mp.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()

Overwriting multiprocessing_test.py


In [11]:
%%writefile multiprocessing_test.py
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    ctx = mp.get_context('spawn')
    q = ctx.Queue()
    p = ctx.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()


Overwriting multiprocessing_test.py


In [1]:
%%writefile multiprocessing_test.py
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()

Overwriting multiprocessing_test.py


In [2]:
%%writefile multiprocessing_test.py
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

Overwriting multiprocessing_test.py


In [3]:
%%writefile multiprocessing_test.py
from multiprocessing import Process, Lock

def f(l, i):
    l.acquire()
    try:
        print('hello world', i)
    finally:
        l.release()

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()

Overwriting multiprocessing_test.py


In [4]:
%%writefile multiprocessing_test.py
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]

if __name__ == '__main__':
    num  = Value('d', 0.0)
    arr = Array('i', range(10))

    p = Process(target=f, args=(num, arr))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])

Overwriting multiprocessing_test.py


In [ ]:
%%writefile multiprocessing_test.py
from multiprocessing import Process, Manager
from time import sleep

def func(id, d):
    print('id: {}, d: {}'.format(id, d))
    sleep(3)
    d[id] = id
    print('id: {}, d: {}'.format(id, d))

def main():
    manager = Manager()
    d = manager.dict()
    p1 = Process(target=func, args=(1, d))
    p2 = Process(target=func, args=(2, d))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(d)

if __name__ == '__main__':
    main()

In [ ]:
%%writefile multiprocessing_test.py
from multiprocessing import Process, Manager
from time import sleep

# try SyncManager.Condition, consumer and producer

def consumer(cond, id):
    """wait for the condition and use the resource"""
    with cond:
        cond.wait()
        print('Consumer {} is consuming'.format(id))

def producer(cond, id):
    """set up the resource to be used by the consumer"""
    with cond:
        print('Producer {} is producing'.format(id))
        cond.notify_all()

if __name__ == '__main__':
    manager = Manager()
    condition = manager.Condition()
    p1 = Process(target=producer, args=(condition, 1))
    p2 = Process(target=producer, args=(condition, 2))
    c1 = Process(target=consumer, args=(condition, 1))
    c2 = Process(target=consumer, args=(condition, 2))
    c3 = Process(target=consumer, args=(condition, 3))

    c1.start()
    sleep(1)
    c2.start()
    sleep(1)
    c3.start()
    sleep(1)
    p1.start()
    sleep(1)
    p2.start()
    sleep(1)

    c1.join()
    c2.join()
    c3.join()
    p1.join()
    p2.join()
    print('All processes done')




In [ ]:
%% writefile multiprocessing_test.py
from multiprocessing import Process, Pool
import time

# show me hao to use the pool to organize the processes

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(processes=4) as pool:        # start 4 worker processes
        result = pool.apply_async(f, (10,)) # evaluate "f(10)" asynchronously in a single process
        print(result.get(timeout=1))        # prints "100" unless your computer is *very* slow

        print(pool.map(f, range(10)))       # prints "[0, 1, 4,..., 81]"

        it = pool.imap(f, range(10))
        print(next(it))                     # prints "0"
        print(next(it))                     # prints "1"
        print(it.next(timeout=1))           # prints "4" unless your computer is *very* slow

        result = pool.apply_async(time.sleep, (10,))
        print(result.get(timeout=1))        # raises multiprocessing.TimeoutError